In [1]:
# imports and setup 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout


Using TensorFlow backend.


In [2]:
def organize_pixel_values(raw_df):
    df = raw_df.copy();
    for i in range(12):
        df.drop(df.columns[[0]], axis=1, inplace=True)

    ret = []
    column_vals = df.columns
    df_matrix = df.to_numpy()
    
    for row in range(len(df_matrix)):
        pixel_matrix = np.zeros((20, 20))
        for col in range(len(df_matrix[0])):
            header = column_vals[col]
            splitHeader = header[1:].split('c')
            pixelRow = splitHeader[0]
            pixelCol = splitHeader[1]
            pixel_matrix[int(pixelRow)][int(pixelCol)] = df_matrix[row][col] / 255.0
        ret.append(pixel_matrix)
    
    ret = np.stack(ret)
    ret = np.reshape(ret, (-1, 20, 20, 1))
    
    return ret

In [3]:
def get_maps_from_labels(input_arr):  
    val_to_ix = { val:i for i,val in enumerate(np.unique(input_arr)) }
    ix_to_val = { i:val for i,val in enumerate(np.unique(input_arr)) }
    
    return val_to_ix, ix_to_val

In [114]:
def reconcileFonts(font1, font2):
#     for i in font1.y
#         if value is not key in font1.ix_to_val
#             remove font1.y[i], font1.x[i]

    font1_keys = set(font1.val_to_ix.keys())
    font2_keys = set(font2.val_to_ix.keys())
    
    print(len(font1_keys), len(font2_keys))
    
    intersection = np.array(font1_keys & font2_keys)
    
    print(intersection.shape)
    print()
    print(font1.y_raw.shape)
    
#     font1.val_to_ix, font1.ix_to_val = get_maps_from_labels(np.array(intersection))
#     font2.val_to_ix, font2.ix_to_val = get_maps_from_labels(np.array(intersection))
    
#     print(len(font1.val_to_ix), len(font2.val_to_ix))
    
    
    
    
#     y_fixed = []
#     x_fixed = []
#     for i in range(len(font1.y_raw)):
#         if (i in font2.ix_to_val):
#             y_fixed.append(font1.y_raw[i])
#             x_fixed.append(font1.x[i])
#             print("Save")
#         else:
#             print ("Delete")
    
#     print(y_fixed)
#     print(get_maps_from_labels(y_fixed)[0])
#     print(len(get_maps_from_labels(y_fixed)[0]))

In [115]:
reconcileFonts(arial, timesNewRoman)

3098 3087
()

(26237,)


In [49]:
class font:
    x = None
    y = None
    y_raw = None
    
    unique_char_count = 0
    val_to_ix = None
    ix_to_val = None
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        
        self.x = organize_pixel_values(df)
        
        self.y_raw = np.array(df.m_label)
        self.val_to_ix, self.ix_to_val = get_maps_from_labels(self.y_raw)
        self.unique_char_count = len(self.val_to_ix)
        
        self.y = self.get_1_hot()
        
    def get_1_hot(self):
        ret = []
        for val in self.y_raw:
            arr = np.zeros(self.unique_char_count)
            arr[self.val_to_ix[val]] = 1;
            ret.append(arr)

        return np.array(ret)
    
    def display_attributes(self):
        print(self.ix_to_val[0])
        print(self.val_to_ix[33])

        print()

        print(self.unique_char_count)

        print()

        print(self.x.shape)
        print(self.y.shape)

In [50]:
arial = font('fonts/ARIAL.csv')

In [42]:
arial.display_attributes()

33
0

3098

(26237, 20, 20, 1)
(26237, 3098)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(arial.x, arial.y, random_state=1, test_size=0.8)

In [8]:
model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(arial.unique_char_count, activation='softmax'))

W0711 14:43:23.030337 4476843456 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 14:43:23.076234 4476843456 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 14:43:23.092101 4476843456 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 14:43:23.126682 4476843456 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0711 14:43:23.165050 4476843456 deprecation_wrapper.py:119] From /usr/loca

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0711 14:43:23.223360 4476843456 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0711 14:43:23.250074 4476843456 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [10]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

W0711 14:43:23.408517 4476843456 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 5s 1ms/step - loss: 6.6695 - acc: 0.1618 - val_loss: 5.9132 - val_acc: 0.2483
Epoch 2/20
5247/5247 [==============================] - 5s 942us/step - loss: 5.1363 - acc: 0.2929 - val_loss: 5.6574 - val_acc: 0.2922
Epoch 3/20
5247/5247 [==============================] - 5s 950us/step - loss: 4.2986 - acc: 0.3208 - val_loss: 5.7233 - val_acc: 0.3297
Epoch 4/20
5247/5247 [==============================] - 5s 938us/step - loss: 3.4136 - acc: 0.3932 - val_loss: 5.5520 - val_acc: 0.3733
Epoch 5/20
5247/5247 [==============================] - 5s 951us/step - loss: 2.6171 - acc: 0.4629 - val_loss: 5.3779 - val_acc: 0.3998
Epoch 6/20
5247/5247 [==============================] - 5s 948us/step - loss: 2.0303 - acc: 0.5186 - val_loss: 5.2678 - val_acc: 0.4192
Epoch 7/20
5247/5247 [==============================] - 6s 1ms/step - loss: 1.6500 - acc: 0.5695 - val_loss: 5.2637 - val_acc: 0.4136
Epo

### Evaluate the network using cross validation (splitting data into training/testing). What is its accuracy? ###

**Training and testing on Arial, our model gives us an accuracy of around 47% after 20 epochs**

In [11]:
model2 = Sequential()

model2.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model2.add(MaxPooling2D((2, 2), padding="same"))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D((2, 2), padding="same"))
model2.add(Flatten())
model2.add(Dropout(.1))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(arial.unique_char_count, activation='softmax'))

In [12]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 10s 2ms/step - loss: 6.4524 - acc: 0.1843 - val_loss: 5.7995 - val_acc: 0.2728
Epoch 2/20
5247/5247 [==============================] - 10s 2ms/step - loss: 4.8385 - acc: 0.3105 - val_loss: 5.4032 - val_acc: 0.3072
Epoch 3/20
5247/5247 [==============================] - 9s 2ms/step - loss: 3.7011 - acc: 0.3694 - val_loss: 5.4557 - val_acc: 0.3639
Epoch 4/20
5247/5247 [==============================] - 11s 2ms/step - loss: 2.6890 - acc: 0.4589 - val_loss: 5.3654 - val_acc: 0.3967
Epoch 5/20
5247/5247 [==============================] - 12s 2ms/step - loss: 1.9660 - acc: 0.5251 - val_loss: 5.3116 - val_acc: 0.4113
Epoch 6/20
5247/5247 [==============================] - 12s 2ms/step - loss: 1.5409 - acc: 0.5860 - val_loss: 5.2043 - val_acc: 0.4285
Epoch 7/20
5247/5247 [==============================] - 12s 2ms/step - loss: 1.2694 - acc: 0.6280 - val_loss: 5.2467 - val_acc: 0.4416
Epoch 8

### Create and train a different network topology (add more convolution/dropout layers, explore other types/sizes of layer). Try to find a topology that works better than the one described above. ###

**I took a very simple approach with my second model, I just doubled the number of perceptrons of the convolutional and dense layers. This new model gives us a modest increase of accuracy (48% after 20 epochs).  Each epoch took between 2 and 2.5x as long to run though, so it's possible that the increased accuracy would not be worth the time/computing cost in practice**

In [14]:
model3 = Sequential()

model3.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=(20, 20, 1)))
model3.add(MaxPooling2D((2, 2), padding="same"))
model3.add(Conv2D(64, kernel_size=5, activation='relu'))
model3.add(MaxPooling2D((2, 2), padding="same"))
model3.add(Flatten())
model3.add(Dropout(.1))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(arial.unique_char_count, activation='softmax'))

In [15]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 5s 975us/step - loss: 6.7030 - acc: 0.1357 - val_loss: 5.9598 - val_acc: 0.2660
Epoch 2/20
5247/5247 [==============================] - 4s 841us/step - loss: 5.0567 - acc: 0.2931 - val_loss: 5.4785 - val_acc: 0.2927
Epoch 3/20
5247/5247 [==============================] - 5s 912us/step - loss: 4.1361 - acc: 0.3307 - val_loss: 5.6373 - val_acc: 0.3394
Epoch 4/20
5247/5247 [==============================] - 5s 873us/step - loss: 3.3083 - acc: 0.3913 - val_loss: 5.4549 - val_acc: 0.3672
Epoch 5/20
5247/5247 [==============================] - 5s 896us/step - loss: 2.5561 - acc: 0.4464 - val_loss: 5.2991 - val_acc: 0.3948
Epoch 6/20
5247/5247 [==============================] - 5s 952us/step - loss: 2.0797 - acc: 0.4904 - val_loss: 5.1689 - val_acc: 0.4126
Epoch 7/20
5247/5247 [==============================] - 5s 908us/step - loss: 1.7754 - acc: 0.5319 - val_loss: 5.1054 - val_acc: 0.4274

In [17]:
model4 = Sequential()

model4.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Conv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Conv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Flatten())
model4.add(Dropout(.1))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(arial.unique_char_count, activation='softmax'))

In [18]:
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model4.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 8s 2ms/step - loss: 6.8842 - acc: 0.0661 - val_loss: 6.2954 - val_acc: 0.1274
Epoch 2/20
5247/5247 [==============================] - 6s 1ms/step - loss: 5.5751 - acc: 0.2323 - val_loss: 5.7533 - val_acc: 0.2794
Epoch 3/20
5247/5247 [==============================] - 5s 1ms/step - loss: 4.7953 - acc: 0.2990 - val_loss: 5.6876 - val_acc: 0.3017
Epoch 4/20
5247/5247 [==============================] - 5s 958us/step - loss: 4.2420 - acc: 0.3206 - val_loss: 5.6034 - val_acc: 0.3216
Epoch 5/20
5247/5247 [==============================] - 5s 969us/step - loss: 3.7094 - acc: 0.3469 - val_loss: 5.7690 - val_acc: 0.3398
Epoch 6/20
5247/5247 [==============================] - 5s 1ms/step - loss: 3.2393 - acc: 0.3806 - val_loss: 5.7219 - val_acc: 0.3510
Epoch 7/20
5247/5247 [==============================] - 6s 1ms/step - loss: 2.8533 - acc: 0.4052 - val_loss: 5.5675 - val_acc: 0.3710
Epoch 8/2

### Third and Fourth Models ###

**I did a third and forth model mostly for my enjoyment (since the second model was already an improvment over the first).  The third model I changed the kernel size to 5x5 and it underpreformed with 47% accuracy.  The fourth model I added one additional convolutional layer * max pooling layer and (to my surprise) it dramatically underperformed at 43% accuracy.**

In [51]:
timesNewRoman = font('fonts/TIMES.csv')

In [40]:
timesNewRoman.display_attributes()

33
0

3087

(12730, 20, 20, 1)
(12730, 3087)


In [52]:
reconcileFonts(arial, timesNewRoman)

3098 3087


AttributeError: 'dict' object has no attribute 'intersection'

In [36]:
model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(arial.unique_char_count, activation='softmax'))

In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
model.fit(arial.x, arial.y, validation_data=(timesNewRoman.x, timesNewRoman.y), epochs=20)

ValueError: Error when checking target: expected dense_14 to have shape (3098,) but got array with shape (3087,)

### Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform? ###

**I trained with Arial and tested on Times New Roman**


Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform? (He means train on one font, test on another)

Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case? What accuracy do you see when testing with inputs from a font you didn't train on?
Take a look at some of the characters that have been misclassified. Do you notice any patterns? The network only produces the relative probabilities that the input is any of the possible characters. Can you find examples where the network is unsure of the result?